In [1]:
#!pip install catboost
#!pip install vecstack

In [2]:
%run import_modules.py

In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\82105\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

# 1. Data Load

In [3]:
X_train = pd.read_csv('train.csv').drop('index', axis=1)
y_train = X_train['voted']
X_train = X_train.drop('voted', axis=1)
X_test = pd.read_csv('test_x.csv').drop('index', axis=1)


# 2. Feature Engineering

## 2-1. 변수 처리

In [4]:

### 이상치제거
def to_int32(dataDF):
    Answers = dataDF.filter(regex='A').columns.tolist()
    
    ### float 데이터 -> int로 처리
    for feature in Answers:
        dataDF[feature] = dataDF[feature].astype('int32') 
    
    dataDF['Q_A_score'] = dataDF[Answers].mean(axis = 1)
    return dataDF

X_train = to_int32(X_train)
X_test = to_int32(X_test)


### 스케일링[RobustScaler]
robustScaler = RobustScaler()
features = X_train.filter(regex='E').columns.tolist()
X_train[features] = robustScaler.fit_transform(np.array(X_train[features])) 
X_test[features] = robustScaler.transform(np.array(X_test[features]))


########## 피처 추가 #############
# 응답 종류 개수
X_train_Q_A = X_train.filter(like = 'A')
X_test_Q_A = X_test.filter(like = 'A')
X_train['Q_A_count'] = X_train_Q_A.nunique(axis = 1)
X_test['Q_A_count'] = X_test_Q_A.nunique(axis = 1)

# 응답평균시간
X_train_Q_E = X_train.filter(like = 'E')
X_test_Q_E = X_test.filter(like = 'E')
X_train['Q_E_mean'] = X_train_Q_E.mean(axis = 1)
X_test['Q_E_mean'] = X_test_Q_E.mean(axis = 1)

# 응답최대시간
X_train['Q_E_max'] = X_train_Q_E.max(axis = 1)
X_test['Q_E_max'] = X_test_Q_E.max(axis = 1)

# wr_(01-13) : 실존하는 해당 단어의 정의을 앎 & wf_(01-03) : 허구인 단어의 정의를 앎
# yes = 1, No = 0
# 합산을 통해서 얼마나 쌈박하게 똑똑한지
X_train_wr = X_train.filter(like = 'wr')
X_test_wr = X_test.filter(like = 'wr')
X_train_wf = X_train.filter(like = 'wf')
X_test_wf = X_test.filter(like = 'wf')
X_train['wr_sum'] = X_train_wr.sum(axis = 1)
X_test['wr_sum'] = X_test_wr.sum(axis = 1)
X_train['wf_sum'] = X_train_wf.sum(axis = 1)
X_test['wf_sum'] = X_test_wf.sum(axis = 1)

# 응답시간 변동계수
X_train['Q_E_cov'] = X_train_Q_E.std(axis = 1)/X_train_Q_E.mean(axis = 1)
X_test['Q_E_cov'] = X_test_Q_E.std(axis = 1)/X_test_Q_E.mean(axis = 1)

# 응답별 응답비율
X_train_total = X_train_Q_E.sum()
X_test_total = X_test_Q_E.sum()
X_train_Q_E_ratio = X_train_Q_E/X_train_total * 100000 # 단위가 너무 낮아서 100000 곱해줌
X_test_Q_E_ratio = X_test_Q_E/X_test_total * 100000
X_train_Q_E_ratio = X_train_Q_E_ratio.add_suffix('_ratio')
X_test_Q_E_ratio = X_test_Q_E_ratio.add_suffix('_ratio')
X_train = pd.concat([X_train, X_train_Q_E_ratio], axis = 1)
X_test = pd.concat([X_test, X_test_Q_E_ratio], axis = 1)



### 인코딩[라벨화]
le = LabelEncoder()
features = ['age_group','gender']
for feature in features:
    X_train[feature] = le.fit_transform(np.array(X_train[feature]))
    X_test[feature] = le.transform(np.array(X_test[feature]))
    
    
### 인코딩[원핫인코딩]
def onehot_cat_features(dataDF):
    
    dataDF = pd.get_dummies(dataDF, columns = ['race', 'religion'])
    
    return dataDF

X_train = onehot_cat_features(X_train)
X_test = onehot_cat_features(X_test)
    
print(X_train.shape)
print(X_test.shape)


(45532, 120)
(11383, 120)


# 3. Modeling

In [5]:
### skf first, sscv later
sscv = StratifiedShuffleSplit(n_splits = 4, random_state=24)
skf = StratifiedKFold(n_splits=4 , shuffle=True, random_state=24)
cv = skf

In [6]:
X_test = X_test.values

In [7]:
models_for_ensemble = [
    ('LGBMClassifier', LGBMClassifier(colsample_bytree=0.7604296531524981,
               learning_rate=0.02307705418401338, max_depth=26,
               min_child_samples=34, min_child_weight=10.666818829880135,
               n_estimators=169, num_leaves=39, objective='binary',
               subsample=0.8577600405357944), 0.7694156640221028),
    ('ExtraTreesClassifier', ExtraTreesClassifier(bootstrap=True, max_depth=14,
                     max_features=0.8094388028799028, min_samples_leaf=16,
                     min_samples_split=25, n_estimators=161, n_jobs=-1,
                     oob_score=True, random_state=0), 0.7708838436513831)
]

In [8]:
### 사용하지 않을 모델 주석처리
models = [
    'ExtraTreesClassifier',
    'LGBMClassifier',
#    'GradientBoostingClassifier',
#    'XGBClassifier',
#    'CatBoostClassifier'
]

selected_models = [clf for clf in models_for_ensemble if clf[0] in models]
BO_tuned_clfs = selected_models

### 3-1. extra, lgb 기하평균, 가중평균


In [9]:
### set models & submission name

for clf in BO_tuned_clfs:
    
    model_name = clf[0]
    submission_name = '[120features]' + f'{model_name}' + '_12models_gmean.csv'

    ### produce folder & save models
    model_storage = 'pkl_storage2'
    folder_name = model_name
    #folder_name = ', '.join([clf[0] for clf in BO_tuned_clfs])
    if os.path.exists(f'{model_storage}') == False:
        os.mkdir(f'{model_storage}')
    if os.path.exists(f'{model_storage}/{folder_name}') == False:
        os.mkdir(f'{model_storage}/{folder_name}')
        
    ### model을 (seed * n_splits)개 찍어서 gmean

#    averaging = VotingClassifier(
#       estimators = [(clf[0],  clf[1]) for clf in BO_tuned_clfs], voting='soft')
    model = clf[1]

    start = time.time()
    lucky_seed = [1234,24,91]
    for seed in lucky_seed:
        print(f'< {model_name} 학습시작 >')
        skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
        model.random_state = seed
        scores = []
        for iter_count, (train_index, valid_index) in enumerate(skf.split(X_train, y_train)):

            X_tra, X_valid = X_train.values[train_index], X_train.values[valid_index]
            y_tra, y_valid = y_train.values[train_index], y_train.values[valid_index]

            model.fit(X_tra, y_tra)
            joblib.dump(model, f'{model_storage}/{folder_name}/{folder_name}_{iter_count}_{seed}.pkl')        

            pred = model.predict_proba(X_valid)[:,1]
            score = roc_auc_score(y_valid, pred)
            scores.append(score)
            print(f'   {iter_count+1}번째 ROC_AUC {score:.4f}')

        mean_score = np.mean(scores)
        print(f'     평균 정확도: {mean_score:.4f}') 

    print(f'{model_name} 모델 {len(lucky_seed) * skf.n_splits}개 학습 완료!,  걸린시간: {time.time() - start}초')

    ### load models & predict_proba
    models = os.listdir(f'{model_storage}/{folder_name}')
    models_list = [x for x in models if x.endswith(".pkl")]
    assert len(models_list) == 12 # 12개 아니면 error

    ### gmean
    preds = []
    for m in models_list:
        model = joblib.load(f'{model_storage}/{folder_name}/'+m)
        predict_proba = model.predict_proba(X_test)[:,1]
        preds.append(predict_proba)
    pred_mean = gmean([each_seed_pred for each_seed_pred in preds])
    sub = pd.DataFrame({'index':range(11383), 'voted':pred_mean})
    sub.to_csv(f'기하평균 {submission_name}', index=False)
    print('기하평균 csv 파일 출력 완료!')


< LGBMClassifier 학습시작 >
   1번째 ROC_AUC 0.7682
   2번째 ROC_AUC 0.7667
   3번째 ROC_AUC 0.7706
   4번째 ROC_AUC 0.7671
     평균 정확도: 0.7681
< LGBMClassifier 학습시작 >
   1번째 ROC_AUC 0.7673
   2번째 ROC_AUC 0.7682
   3번째 ROC_AUC 0.7737
   4번째 ROC_AUC 0.7658
     평균 정확도: 0.7687
< LGBMClassifier 학습시작 >
   1번째 ROC_AUC 0.7683
   2번째 ROC_AUC 0.7736
   3번째 ROC_AUC 0.7683
   4번째 ROC_AUC 0.7668
     평균 정확도: 0.7693
LGBMClassifier 모델 12개 학습 완료!,  걸린시간: 20.169060230255127초
기하평균 csv 파일 출력 완료!
< ExtraTreesClassifier 학습시작 >
   1번째 ROC_AUC 0.7711
   2번째 ROC_AUC 0.7698
   3번째 ROC_AUC 0.7713
   4번째 ROC_AUC 0.7690
     평균 정확도: 0.7703
< ExtraTreesClassifier 학습시작 >
   1번째 ROC_AUC 0.7689
   2번째 ROC_AUC 0.7709
   3번째 ROC_AUC 0.7742
   4번째 ROC_AUC 0.7660
     평균 정확도: 0.7700
< ExtraTreesClassifier 학습시작 >
   1번째 ROC_AUC 0.7701
   2번째 ROC_AUC 0.7744
   3번째 ROC_AUC 0.7687
   4번째 ROC_AUC 0.7674
     평균 정확도: 0.7702
ExtraTreesClassifier 모델 12개 학습 완료!,  걸린시간: 149.17559218406677초
기하평균 csv 파일 출력 완료!


In [10]:
### 모델별 gmean된 파일을 불러와서 평균
extra_gmean = pd.read_csv('기하평균 [120features]ExtraTreesClassifier_12models_gmean.csv').voted
lgb_gmean = pd.read_csv('기하평균 [120features]LGBMClassifier_12models_gmean.csv').voted
submission_name = '[120features] extra, lgb.csv'

### 가중평균
extra_weight = 0.675; lab_weight = 0.325;
pred_mean = (extra_gmean * extra_weight) + (lgb_gmean * lab_weight)
sub = pd.DataFrame({'index':range(X_test.shape[0]), 'voted':pred_mean})
sub.to_csv(f'가중평균 {submission_name}', index=False)
print('가중평균 출력완료!')

### 기하평균
pred_lst = [extra_gmean, lgb_gmean]
pred_mean = gmean(pred_lst)
sub = pd.DataFrame({'index':range(X_test.shape[0]), 'voted':pred_mean})
sub.to_csv(f'기하평균 {submission_name}', index=False) 
print('기하평균 출력완료!')

가중평균 출력완료!
기하평균 출력완료!


In [11]:
#best1_120feature = pd.read_csv('가중평균 [c~AF]extra0.675 + lgb0.325.csv').rename(columns={'voted':'120features'})['120features']
#best2_94feature = pd.read_csv('가중평균 [94 features + binning 다르게] extra, lgb.csv').rename(columns={'voted':'94features'})['94features']
#best3_155feature = pd.read_csv('기하평균 [1104 + pca] extra, lgb.csv').rename(columns={'voted':'155features'})['155features']

#compare_features = [best1_120feature, best2_94feature, best3_155feature]
#pd.DataFrame(compare_features).T.corr()

GMEAN

120features : 0.780688 (현 베스트)   
94 features + binning :0.780556    
155 features = 1104 + pca : 0.780497

In [12]:
#best1_120feature = pd.read_csv('가중평균 [c~AF]extra0.675 + lgb0.325.csv').voted
#best2_94feature = pd.read_csv('가중평균 [94 features + binning 다르게] extra, lgb.csv').voted
#best3_155feature = pd.read_csv('기하평균 [1104 + pca] extra, lgb.csv').voted

#submission_name = '[120, 94, 155 features] extra, lgb.csv'

#preds = [best1_120feature, best2_94feature, best3_155feature]
#pred_mean = gmean(preds)
#sub = pd.DataFrame({'index':range(X_test.shape[0]), 'voted':pred_mean})
#sub.to_csv(f'기하평균 {submission_name}', index=False) 
#print('기하평균 출력완료!')
